<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Data-API" data-toc-modified-id="Data-Data-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Data API</a></span><ul class="toc-item"><li><span><a href="#Chaining-Transformations" data-toc-modified-id="Chaining-Transformations-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Chaining Transformations</a></span></li><li><span><a href="#Shuffling-the-Data" data-toc-modified-id="Shuffling-the-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Shuffling the Data</a></span><ul class="toc-item"><li><span><a href="#Interleaving-lines-from-multiple-files" data-toc-modified-id="Interleaving-lines-from-multiple-files-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Interleaving lines from multiple files</a></span></li></ul></li><li><span><a href="#Preprocessing-the-Data" data-toc-modified-id="Preprocessing-the-Data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Preprocessing the Data</a></span></li><li><span><a href="#Putting-Everything-Together" data-toc-modified-id="Putting-Everything-Together-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Putting Everything Together</a></span></li><li><span><a href="#Prefetching" data-toc-modified-id="Prefetching-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Prefetching</a></span></li><li><span><a href="#Using-the-Dataset-with-tf.keras" data-toc-modified-id="Using-the-Dataset-with-tf.keras-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Using the Dataset with tf.keras</a></span></li></ul></li><li><span><a href="#The-TFRecord-Format" data-toc-modified-id="The-TFRecord-Format-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The TFRecord Format</a></span><ul class="toc-item"><li><span><a href="#Compressed-TFRecord-Files" data-toc-modified-id="Compressed-TFRecord-Files-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Compressed TFRecord Files</a></span></li><li><span><a href="#A-Brief-Introduction-to-Protocol-Buffers-(-protobufs-)" data-toc-modified-id="A-Brief-Introduction-to-Protocol-Buffers-(-protobufs-)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>A Brief Introduction to Protocol Buffers ( <em>protobufs</em> )</a></span></li><li><span><a href="#TensorFlow-Protobufs" data-toc-modified-id="TensorFlow-Protobufs-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>TensorFlow Protobufs</a></span></li><li><span><a href="#Loading-and-Parsing-Examples" data-toc-modified-id="Loading-and-Parsing-Examples-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Loading and Parsing Examples</a></span></li><li><span><a href="#Handling-Lists-of-Lists-Using-the-SequenceExample-Protobuf" data-toc-modified-id="Handling-Lists-of-Lists-Using-the-SequenceExample-Protobuf-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Handling Lists of Lists Using the SequenceExample Protobuf</a></span></li></ul></li><li><span><a href="#Preprocessing-the-Input-Features" data-toc-modified-id="Preprocessing-the-Input-Features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocessing the Input Features</a></span><ul class="toc-item"><li><span><a href="#Encoding-Categorical-Features-Using-One-Hot-Vectors" data-toc-modified-id="Encoding-Categorical-Features-Using-One-Hot-Vectors-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Encoding Categorical Features Using One-Hot Vectors</a></span></li><li><span><a href="#Encoding-Categorical-Features-Using-Embeddings" data-toc-modified-id="Encoding-Categorical-Features-Using-Embeddings-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Encoding Categorical Features Using Embeddings</a></span></li><li><span><a href="#Keras-Preprocessing-Layers" data-toc-modified-id="Keras-Preprocessing-Layers-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Keras Preprocessing Layers</a></span></li></ul></li><li><span><a href="#TF-Transform" data-toc-modified-id="TF-Transform-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TF Transform</a></span></li><li><span><a href="#The-TensorFlow-Datasets(TFDS)-Project" data-toc-modified-id="The-TensorFlow-Datasets(TFDS)-Project-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>The TensorFlow Datasets(TFDS) Project</a></span></li></ul></div>

In [1]:
import tensorflow as tf
from tensorflow import keras

## Data Data API

In [2]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [3]:
for data in dataset:
    print(data)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


### Chaining Transformations

In [5]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [13]:
dataset = dataset.map(lambda x: tf.cast(x, dtype=tf.float32)**0.5, num_parallel_calls=4)

In [14]:
for item in dataset:
    print(item)

tf.Tensor([ 0.  1.  4.  9. 16. 25. 36.], shape=(7,), dtype=float32)
tf.Tensor([49. 64. 81.  0.  1.  4.  9.], shape=(7,), dtype=float32)
tf.Tensor([16. 25. 36. 49. 64. 81.  0.], shape=(7,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25. 36. 49.], shape=(7,), dtype=float32)
tf.Tensor([64. 81.], shape=(2,), dtype=float32)


In [34]:
dataset = dataset.unbatch()

In [41]:
for item in dataset.take(2):
    print(item)

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [38]:
dataset.filter(lambda x: x < 10)

<FilterDataset shapes: (), types: tf.float32>

In [39]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


### Shuffling the Data

In [47]:
dataset = tf.data.Dataset.range(10).repeat(3)

In [48]:
# shufful only take elements from buffer
# and buffer will filled by next first element from dataset
# so when we set buffer_size=1, the shuffle make no sense
dataset = dataset.shuffle(buffer_size=1, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [49]:
# if we change buffer_size=2
# every time the dataset object will pick one element from buffer,
# and place a new element from dataset on it immediately
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=2, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([1 2 0 3 5 4 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 0 9 1 3 4], shape=(7,), dtype=int64)
tf.Tensor([5 2 7 6 9 0 8], shape=(7,), dtype=int64)
tf.Tensor([2 1 4 3 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([9 8], shape=(2,), dtype=int64)


So, in the scenario above (with buffer=2):  
0, 1 -> pick 1  
0, 2 -> pick 2  
0, 3 -> pick 0  
4, 3 -> pick 3  
4, 5 -> pick 5  
4, 6 -> pick 4  
7, 6 -> pick 6  
......

#### Interleaving lines from multiple files

In [50]:
filepath_dataset = tf.data.Dataset.list_files("/tf/notebooks/data/*.csv", shuffle=False, seed=42)

In [139]:
# explaination of interleave():
# 假定我们现在有一个Dataset--A
# 从该A中取出cycle_length个element，然后对这些element apply map_func,得到cycle_length个新的Dataset对象。
# 然后从这些新生成的Dataset对象中取数据，取数逻辑为轮流从每个对象里面取数据，每次取block_length个数据
# 当这些新生成的某个Dataset的对象取尽时，从原Dataset中再取cycle_length个element，，然后apply map_func，以此类推。

a = tf.data.Dataset.range(1, 6)  # ==> [ 1, 2, 3, 4, 5 ]
b = a.interleave(lambda x: tf.data.Dataset.from_tensors(x).repeat(5),
            cycle_length=3, block_length=2) 
for item in b:
    print(item.numpy(),end=', ')

1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3, 1, 2, 3, 4, 4, 5, 5, 4, 4, 5, 5, 4, 5, 

In [59]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers
)

### Preprocessing the Data

In [85]:
def preporcess(line):
    defs = [0.] * 8 + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    return tf.stack(fields[:-1]), tf.stack(fields[-1:])

In [86]:
preporcess(b'4, 10, 12, 25, 5523, 123, 213,, 10')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([4.000e+00, 1.000e+01, 1.200e+01, 2.500e+01, 5.523e+03, 1.230e+02,
        2.130e+02, 0.000e+00], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([10.], dtype=float32)>)

### Putting Everything Together

In [88]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5, 
                       n_read_threads=None, shuffle_buffer_size=10000, 
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths, shuffle=True, seed=42)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepaths).skip(1), 
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preporcess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(buffer_size=shuffle_buffer_size).repeat(repeat)
    return dataset

### Prefetching

### Using the Dataset with tf.keras

In [ ]:
train_set = csv_reader_dataset(train_filepaths)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [142]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.l2(0.05), input_shape=[30])
])

In [144]:
model.compile(loss=keras.losses.mse, optimizer=keras.optimizers.Adam(0.005))

In [ ]:
model.fit(train_set, epochs=10, validation_data=valid_set)

In [ ]:
# or we use custom training loop
for X_batch, y_batch in train_set:
    [...]

In [149]:
# other way is to create tf.function function to train
@tf.function
def train(model, optimizer, loss_fn, n_epochs, **kwargs):
    train_set = csv_reader_dataset(train_filepath, repeat=n_epochs, **kwargs)
    for X_batch, y_batch in train_set:
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

## The TFRecord Format

In [150]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"This is the second record")

In [154]:
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'This is the second record', shape=(), dtype=string)


### Compressed TFRecord Files

In [155]:
options = tf.io.TFRecordOptions(compression_type='GZIP')
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"Test compression funcionality")

In [156]:
dataset = tf.data.TFRecordDataset('my_compressed.tfrecord', compression_type='GZIP')

In [157]:
for item in dataset:
    print(item)

tf.Tensor(b'Test compression funcionality', shape=(), dtype=string)


### A Brief Introduction to Protocol Buffers ( _protobufs_ )

### TensorFlow Protobufs

In [158]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

In [159]:
person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }))

In [161]:
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializePartialToString())

### Loading and Parsing Examples

In [172]:
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string)
}

In [173]:
for serialized_example in tf.data.TFRecordDataset(['my_contacts.tfrecord']):
    parsed_example = tf.io.parse_single_example(serialized_example, feature_description)

In [175]:
parsed_example["emails"]

In [174]:
tf.sparse.to_dense(parsed_example["emails"], default_value=b"")

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [176]:
parsed_example["emails"].values

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [177]:
# batch parsing
for serialized_example in tf.data.TFRecordDataset(['my_contacts.tfrecord']).repeat(20).batch(10):
    parsed_example = tf.io.parse_example(serialized_example, feature_description)

In [184]:
parsed_example['name']

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'Alice', b'Alice', b'Alice', b'Alice', b'Alice', b'Alice',
       b'Alice', b'Alice', b'Alice', b'Alice'], dtype=object)>

### Handling Lists of Lists Using the SequenceExample Protobuf

## Preprocessing the Input Features

In [ ]:
# use Lambda layer to customize standard deviation preprocess
means = np.mean(X_train, axis=0, keepdims=True)
stds = np.sstd(X_train, axis=0, keepdims=True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([
    keras.layers.Lambda(lambda inputs: (inputs - means) / (stds + eps)),
    [...]
])

In [188]:
# self-contained layer
class Standardization(keras.layers.Layer):
    def apdat(self, data_sample):
        sefl.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

In [189]:
std_layer = Standardization()

### Encoding Categorical Features Using One-Hot Vectors

In [195]:
vocab = ['<1H OCEAN', 'INLAND', "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

In [201]:
# test the loopup table
categories = tf.constant(["<1H OCEAN", 'NEAR BAY', "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 3, 5, 1, 1])>

In [202]:
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
cat_one_hot

<tf.Tensor: shape=(5, 7), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

### Encoding Categorical Features Using Embeddings

In [203]:
embedding_dim = 2
embed_init = tf.random.uniform([len(vocab)+num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.16258681, 0.07541919],
       [0.8378128 , 0.8445947 ],
       [0.15343809, 0.19920743],
       [0.6615534 , 0.21533203],
       [0.6658559 , 0.28871882],
       [0.181041  , 0.29205155],
       [0.7755804 , 0.11226654]], dtype=float32)>

In [205]:
categories = tf.constant(['NEAR BAY', "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1])>

In [206]:
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0.6615534 , 0.21533203],
       [0.181041  , 0.29205155],
       [0.8378128 , 0.8445947 ],
       [0.8378128 , 0.8445947 ]], dtype=float32)>

In [207]:
# use keras
embedding = keras.layers.Embedding(input_dim=len(vocab)+num_oov_buckets, output_dim=embedding_dim)
embedding(cat_indices)

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[ 0.01041894,  0.02498868],
       [ 0.00501535, -0.00349748],
       [-0.0422516 ,  0.04752635],
       [-0.0422516 ,  0.04752635]], dtype=float32)>

A model take regular inputs(numeric) and categorical inputs

In [208]:
# regualr inputs and categorical inputs
regular_inputs = keras.layers.Input(shape=[8,])
categories = keras.layers.Input([], dtype=tf.string)
# index & embedding layers
cat_indices = keras.layers.Lambda(lambda cate: table.lookup(cate))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
# concatenate two inputs
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
# output
outputs = keras.layers.Dense(1)(encoded_inputs)
# model
model = keras.models.Model(inputs=[regular_inputs, categories], outputs=[outputs])

In [209]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None,)              0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 2)            12          lambda[0][0]                     
______________________________________________________________________________________________

### Keras Preprocessing Layers

## TF Transform

## The TensorFlow Datasets(TFDS) Project